In [2]:
import os
import matplotlib.pyplot as plt

%matplotlib inline

In [119]:
import numpy as np
import re
from collections import defaultdict
from itertools import groupby
from collections import Counter
from operator import add
from functools import reduce
import multiprocessing

np.set_printoptions(threshold=np.nan)

tokens = ['com', 'cart', 'buy', 'img', 'h1', 'h2', 'h3', 'h4', 'div', 'span', 'summary',
          'description', 'product', 'shop', 'captcha', 'status', 'amazon', 'waitrose']

rx = re.compile(r'\b(%s)\b' % "|".join(tokens), re.IGNORECASE)

def getfiles(folder):
    return [r + "/" + f for r, s, fs in os.walk(folder) for f in fs]
            
def getCounter(file):
    with open(file) as f:
        cl = [(key, len(list(group))) for line in f for key, group in groupby(rx.findall(line.lower()))]
        return defaultdict(int, reduce(add, (Counter(dict([x])) for x in cl)))
    
def getCounts(files):
    map = defaultdict(lambda: defaultdict(int))
    pool = multiprocessing.Pool(processes = 8)
    counters = pool.map(getCounter, files)
#     counters = [getCounter(file) for file in files]
    return [[counter[token] for token in tokens] for counter in counters]
           
files = getfiles("/home/maghams/tmp/HTMLPage-Classifier-Dataset/html_sample_dataset/all")
X = np.array(getCounts(files))
print(X.shape)

(12036, 18)


In [134]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn import preprocessing
X_scaled = X * 1.0
X_scaled[:,14] *= 1000
X_scaled[:,17] *= 20
y_pred = KMeans(n_clusters=32, random_state=170).fit_predict(X_scaled)

In [135]:
sorted((len(np.where(y_pred == i)[0]), i) for i in range(0, 32))

[(1, 23),
 (2, 8),
 (3, 12),
 (5, 24),
 (7, 5),
 (33, 14),
 (49, 17),
 (59, 18),
 (73, 31),
 (95, 25),
 (121, 4),
 (128, 28),
 (137, 19),
 (159, 30),
 (171, 26),
 (178, 21),
 (193, 15),
 (261, 29),
 (264, 10),
 (265, 7),
 (409, 6),
 (417, 27),
 (543, 3),
 (587, 16),
 (643, 22),
 (729, 2),
 (756, 11),
 (828, 20),
 (1036, 9),
 (1087, 13),
 (1210, 1),
 (1587, 0)]

In [133]:
from subprocess import call
j = 0
for i in np.array(files)[np.where(y_pred == 8)]:
    if ((j%1) == 0):
        call(['firefox', i])
    j += 1